In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from time import time
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mutual_info_score
from math import log2
from scipy.stats import entropy
from scipy import stats


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from time import time
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

import warnings
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [2]:
train = pd.read_csv("./Datasets/Customer_Segmentation_4(20A)/customer_segmentation_data.csv")
data = train.copy()
print(train.shape)
train.head()

(53503, 20)


,Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
0,84966,23,Female,Married,Associate Degree,Mizoram,Entrepreneur,70541,policy5,04-10-2018,Phone,policy2,366603,2749,Group,Email,In-Person Meeting,Afternoon,English,Segment5
1,95568,26,Male,Widowed,Doctorate,Goa,Manager,54168,policy5,11-06-2018,Chat,policy1,780236,1966,Group,Mail,In-Person Meeting,Morning,French,Segment5
2,10544,29,Female,Single,Associate Degree,Rajasthan,Entrepreneur,73899,policy5,06-05-2021,Email,policy3,773926,4413,Group,Email,Mail,Evening,German,Segment3
3,77033,20,Male,Divorced,Bachelor's Degree,Sikkim,Entrepreneur,63381,policy5,09-02-2018,Chat,policy2,787815,4342,Family,Text,In-Person Meeting,Anytime,French,Segment3
4,88160,25,Female,Separated,Bachelor's Degree,West Bengal,Manager,38794,policy1,09-10-2018,Chat,policy4,366506,1276,Family,Email,Text,Weekends,English,Segment2


In [3]:
train = train.dropna()
t_data = train.copy()
# t_data = train.drop(["Customer ID"], axis='columns').copy()
label_encoders = {}
for col in t_data.columns:
    label_encoders[col] = LabelEncoder()
    t_data[col] = label_encoders[col].fit_transform(t_data[col])

t_data

,Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
0,34106,5,0,1,0,22,3,18002,4,270,4,1,19779,2212,2,0,1,0,0,4
1,39052,8,1,4,2,10,5,11755,4,942,0,0,41919,1429,2,2,1,3,1,4
2,3692,11,0,3,0,27,3,19284,4,387,1,2,41396,3876,2,0,2,2,2,2
3,30847,2,1,0,1,28,3,15250,4,582,0,1,42531,3805,1,4,1,1,1,2
4,35939,7,0,2,1,34,5,6147,0,630,0,3,19772,740,1,0,4,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53498,17156,17,0,0,0,0,7,34335,2,1,3,0,31630,3867,1,1,3,0,2,4
53499,24571,43,1,3,2,13,3,35039,4,1569,3,3,34550,749,2,4,2,0,2,0
53500,9521,7,1,0,2,3,8,9555,1,1068,2,0,34199,3870,3,4,4,4,1,3
53501,23258,24,1,1,2,1,0,16169,3,381,2,0,38419,3945,0,2,3,3,1,4


In [4]:
x= t_data.drop("Segmentation Group", axis='columns').copy()
y= t_data['Segmentation Group'].copy()
sp = train_test_split(x,y,test_size=0.33, random_state=11)
x_train = sp[0]
x_test = sp[1]
y_train = sp[2]
y_test = sp[3]

Naive Bayes

In [5]:
naiveBayes = GaussianNB()
naiveBayes.fit(x_train,y_train)
naiveBayes_y_pred = naiveBayes.predict(x_test)
nb_result = classification_report(y_test, naiveBayes_y_pred, output_dict=True)
print(f"Accuracy: {(nb_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(nb_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(nb_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(nb_result.get('weighted avg').get('f1-score')):.3f}")

Accuracy: 26.82%
Precision: 0.226
Recall: 0.268
F1-Score: 0.176


Decision Tree

In [6]:
decision_tree = DecisionTreeClassifier(max_leaf_nodes=5)
decision_tree.fit(x_train,y_train)
decision_tree_y_pred = decision_tree.predict(x_test)
dt_result = classification_report(y_test, decision_tree_y_pred, output_dict=True)
print(f"Accuracy: {(dt_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(dt_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(dt_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(dt_result.get('weighted avg').get('f1-score')):.3f}")
warnings.filterwarnings('ignore')

Accuracy: 26.10%
Precision: 0.164
Recall: 0.261
F1-Score: 0.175


C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest

In [7]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train,y_train)
random_forest_y_pred = random_forest.predict(x_test)
rf_result = classification_report(y_test, random_forest_y_pred, output_dict=True)
print(f"Accuracy: {(rf_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(rf_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(rf_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(rf_result.get('weighted avg').get('f1-score')):.3f}")

Accuracy: 25.02%
Precision: 0.227
Recall: 0.250
F1-Score: 0.218


Bagging

In [8]:
start = time()
bagging = BaggingClassifier(n_estimators=50)
bagging.fit(x_train, y_train)
bagging_y_pred = bagging.predict(x_test)
bagging_result = classification_report(y_test, bagging_y_pred, output_dict=True)
print(f"Accuracy: {(bagging_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(bagging_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(bagging_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(bagging_result.get('weighted avg').get('f1-score')):.3f}")
print(f"Run Time: {time()-start:.3f}")

Accuracy: 23.77%
Precision: 0.222
Recall: 0.238
F1-Score: 0.220
Run Time: 21.136


AdaBoost

In [9]:
start = time()
adaboost = AdaBoostClassifier(n_estimators=200, random_state=1)
adaboost.fit(x_train, y_train)
adaboost_y_pred = adaboost.predict(x_test)
adaboost_result = classification_report(y_test, adaboost_y_pred, output_dict=True)
print(f"Accuracy: {(adaboost_result.get('accuracy')*100):.2f}%")
print(f"Precision: {(adaboost_result.get('weighted avg').get('precision')):.3f}")
print(f"Recall: {(adaboost_result.get('weighted avg').get('recall')):.3f}")
print(f"F1-Score: {(adaboost_result.get('weighted avg').get('f1-score')):.3f}")
print(f"Run Time: {time()-start:.3f}")
warnings.filterwarnings('ignore')

Accuracy: 28.27%
Precision: 0.266
Recall: 0.283
F1-Score: 0.237
Run Time: 11.262
